# Deep Q-Learning for Lunar Landing

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [65]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


### Importing the libraries

In [66]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [67]:
#Network Class
# stateSize is an 8-dimensional vector of each state
  # coordinates of the lander in x & y, linear velocities in x & y, angle, angular velocity, two booleans that represent whether each leg is in contact with the ground or not.
# actionSize is the number of actions available.
  # 0 Do nothing, 1 Fire left orientation engine, 2 Fire main engine, 3 Fire right orientation engine
# seed sets the random seed
# fc1 is the connection between the first layer and the first fully connected layer
# fc2 is connection between first fully connected layer and second
# fc3 is connectio between second fully connected layer and output layer
class Network(nn.Module):
  def __init__(self, stateSize, actionSize, seed = 42):
      super(Network, self).__init__()
      self.seed = torch.manual_seed(seed)
      self.fc1 = nn.Linear(stateSize, 64)
      self.fc2 = nn.Linear(64, 64)
      self.fc3 = nn.Linear(64, actionSize)

  # This function forward propogates the signal from input, through fully connected layers, and to the output
  def forward(self, state):
    # x = F.relu(self.fc1(state)) # This line propogates signal from input to first fully connected layer via the rectified linear unit function.
    # x = F.relu(self.fc2(x))
    # return F.relu(self.fc3(x))
    x = self.fc1(state)
    x = F.relu(x) # This line propogates signal from input to first fully connected layer via the rectified linear unit function.
    x = self.fc2(x)
    x = F.relu(x)
    return self.fc3(x)

## Part 2 - Training the AI

### Setting up the environment

In [68]:
import gymnasium
env = gymnasium.make("LunarLander-v2")
stateShape = env.observation_space.shape    #The vector containing our 8 dimensions
stateSize = env.observation_space.shape[0]  #The number of dimensions of each state
numberActions = env.action_space.n          #The number of actions available
print('State Shape: ', stateShape)
print('State Size: ', stateSize)
print('Number of Actions: ', numberActions)

State Shape:  (8,)
State Size:  8
Number of Actions:  4


### Initializing the hyperparameters

In [69]:
learningRate = 5e-4           #Determined after experimentation
miniBatchSize = 100           #Standard batch size
discountFactor = 0.99          #A gamma closer to 1 will prefer consideration of future rewards. Closer to 0 becomes more short-sighted
replayBufferSize = int(1e5)   #Memory of the AI
interpolationParameter = 1e-3 #Determined after experimentation

### Implementing Experience Replay

In [70]:
class ReplayMemory(object):
  def __init__(self, capacity):
     self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #Enables GPU learning. Uses CUDA cores
     self.capacity = capacity #The number of memories to keep
     self.memory = []         #Array of memories

  # Add a new memory, and delete the oldest memory if capacity is exceeded
  def push(self, event):
    self.memory.append(event)
    if len(self.memory) > self.capacity:
      del self.memory[0]

  def sample(self, batchSize):
    experiences = random.sample(self.memory, k = batchSize) #Get a random sampling of previous memories, equal to the batch size
    # Stack up the states, actions, rewards, next states, and dones into pyTorch Tensors. Tensors are basically matrices for learning application.
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device) #for every state "e[0]" in each experience, add them to a vstack, and convert the numPy arrays to pyTorch tensors.
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    nextStates = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device) # np.uint8 sets this type to bool before converting to float
    return states, nextStates, actions, rewards, dones

### Implementing the DQN class

In [71]:
# The agent that interacts with our lunar landing space environment using deep-Q network. The agent maintains 2 q networks, the local and target.
# The local Q network selects the actions, and the target Q network calculates the target Q values that will be used in the training of the local network
class Agent():
  def __init__(self, stateSize, actionSize):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #Enables GPU learning. Uses CUDA cores
    self.stateSize = stateSize
    self.actionSize = actionSize
    self.localQNetwork = Network(stateSize, actionSize).to(self.device)
    self.targetQNetwork = Network(stateSize, actionSize).to(self.device)
    self.optimizer = optim.Adam(self.localQNetwork.parameters(), lr = learningRate)
    self.memory = ReplayMemory(replayBufferSize)
    self.timeStep = 0

  #Every 4 steps preform a learning. Do so only if we have a number of memories greater than our miniBatchSize.
  def step(self, state, action, reward, nextState, done):
    self.memory.push((state, action, reward, nextState, done))
    self.timeStep = (self.timeStep + 1) % 4
    if self.timeStep == 0:
      if len(self.memory.memory) > miniBatchSize:
        experiences = self.memory.sample(miniBatchSize)
        self.learn(experiences, discountFactor)

  # Selects an action based on a given state and a certain epsilon value
  # Takes into consideration the current policy, epsilon greedy
  def act(self, state, epsilon = 0.):
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    self.localQNetwork.eval()
    with torch.no_grad():
      actionValues = self.localQNetwork(state) #Disables "Gradient Computation"
    self.localQNetwork.train()
    if random.random() > epsilon:
      return np.argmax(actionValues.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.actionSize))

  # Updates updates Q values based on sampled experiences
  def learn(self, experiences, discountFactor):
    states, nextStates, actions, rewards, dones = experiences   # Unpack experiences into sample values
    nextQTargets = self.targetQNetwork(nextStates).detach().max(1)[0].unsqueeze(1)  # Maximum curated values for the next state. Detach is used to detach the tensor from the graph to prevent tracking gradients during backward propagation
    qTargets = rewards + discountFactor * nextQTargets * (1-dones)  # Compute Q Targets for our current state
    qExpected = self.localQNetwork(states).gather(1, actions) # Get expected Q Values from local network
    loss = F.mse_loss(qExpected, qTargets) # Calculated the loss between the expected Q values and the Q Targets
    self.optimizer.zero_grad() # Zero out the optimizer
    loss.backward() #Back propagate this loss
    self.optimizer.step()
    self.softUpdate(self.localQNetwork, self.targetQNetwork, interpolationParameter)

  # Softly updates target parameters using weighted average of local and target parameters.
  # This slow updated prevents large changes from occurring, which overall stabilizes the learning of the model.
  def softUpdate(self, localModel, targetModel, interpolationParameter):
    for targetParameters, localParameters in zip(targetModel.parameters(), localModel.parameters()): # loop through target and local parameters. Use zip to get both at the same time.
      targetParameters.data.copy_(interpolationParameter * localParameters.data + (1.0 - interpolationParameter) * targetParameters.data)



### Initializing the DQN agent

In [72]:
agent = Agent(stateSize, numberActions)

### Training the DQN agent

In [76]:
numberEpisodes = 2000
maximumTimeStepsPerEpisode = 1000
epsilonStartingValue = 1.0
epsilonEndingValue = 0.01
epsilonDecayValue = 0.995 # By setting a marginal decay value, we can allow for different levels of experimentation in the greedy policy.
epsilon = epsilonStartingValue
scoresOn100Episodes = deque(maxlen = 100)

for episode in range(1, numberEpisodes + 1): # The upper bound is excluded in python, so add a +1
  state, _ = env.reset() # Reset the environment with an initial state. Discard anything else besides state with _
  score = 0
  for t in range(maximumTimeStepsPerEpisode):
    action = agent.act(state, epsilon) # Choose an action for this step
    nextState, reward, done, _, _ = env.step(action)  # Run this action through the environment
    agent.step(state, action, reward, nextState, done)  # Train the agent on the ouput from the environment
    state = nextState
    score += reward
    if done:
      break
  scoresOn100Episodes.append(score)
  epsilon = max(epsilonEndingValue, epsilonDecayValue * epsilon)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scoresOn100Episodes)), end="")
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scoresOn100Episodes)))
  if np.mean(scoresOn100Episodes) >= 200.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode, np.mean(scoresOn100Episodes)))
    torch.save(agent.localQNetwork.state_dict(), 'checkpoint.pth')
    break

Episode 100	Average Score: -97.21
Episode 200	Average Score: -18.14
Episode 300	Average Score: 85.47
Episode 400	Average Score: 193.33
Episode 430	Average Score: 200.32
Environment solved in 430 episodes!	Average Score: 200.32


## Part 3 - Visualizing the results

In [81]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env_name):
    env = gymnasium.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v2')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()